# Agente IA para Procesamiento de PDFs desde Gmail y Análisis con ChatGPT


## 🔐 Paso 1: Autenticación con Google Drive

usando cuenta de servicio https://console.cloud.google.com/iam-admin/serviceaccounts/details/107974642318541520307?project=agente-pepe

In [ ]:
#!pip install google-api-python-client google-auth google-auth-oauthlib pdfplumber openai pandas tqdm
#!pip install python-dotenv

In [13]:
from google.oauth2 import service_account
from googleapiclient.discovery import build
import os
import io
from googleapiclient.http import MediaIoBaseDownload
from dotenv import load_dotenv
load_dotenv()
import openai
import json
import sys
import pdfplumber
import pandas as pd
from googleapiclient.discovery import build

# Ruta al archivo de credenciales (descargado desde Google Cloud Console)
SERVICE_ACCOUNT_FILE = os.getenv('SERVICE_ACCOUNT_FILE')
if SERVICE_ACCOUNT_FILE is None:
    raise ValueError("❌ La variable de entorno 'SERVICE_ACCOUNT_FILE' no está definida.")
    
SCOPES = ['https://www.googleapis.com/auth/drive.readonly']

# Autenticación y creación del servicio de Google Drive
credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES)
drive_service = build('drive', 'v3', credentials=credentials)


## 📂 Paso 2: Descarga de archivos PDF desde carpeta de Google Drive

El mail de tu cuenta de servicio (que aparece dentro del credentials.json) debe tener acceso al menos de lectura a esa carpeta FOLDER_ID.

In [9]:
# ID de la carpeta en Google Drive (obtenido desde la URL de la carpeta)
FOLDER_ID = os.getenv('FOLDER_ID')
CARPETA_LOCAL = 'pdfs_descargados'
# Crear carpeta local si no existe
os.makedirs(CARPETA_LOCAL, exist_ok=True)

def descargar_pdfs(folder_id):
    query = f"'{folder_id}' in parents and mimeType='application/pdf'"
    results = drive_service.files().list(q=query, fields="files(id, name)").execute()
    archivos = results.get('files', [])
    nuevos = []

    for archivo in archivos:
        nombre = archivo['name']
        file_id = archivo['id']
        ruta_local = os.path.join(CARPETA_LOCAL, nombre)

        if not os.path.exists(ruta_local):
            request = drive_service.files().get_media(fileId=file_id)
            fh = io.FileIO(ruta_local, 'wb')
            downloader = MediaIoBaseDownload(fh, request)
            done = False
            while not done:
                status, done = downloader.next_chunk()
            print(f"✅ Descargado: {nombre}")
            nuevos.append(nombre)
        else:
            print(f"⚠️ Ya existe: {nombre}")
    return nuevos

# Ejecutar la descarga y obtener solo los nuevos archivos
descargados_nuevos = descargar_pdfs(FOLDER_ID)


✅ Descargado: factura-prueba-uytech.pdf


## 📄 Paso 3: Extracción de texto desde los PDFs descargados

- Usa $\text{pdfplumber}$, que es excelente para leer texto de PDFs "normales".
- Abre un archivo PDF y concatena el texto de todas sus páginas.
- Muestra los primeros 2000 caracteres del texto extraído para que puedas inspeccionarlo.

In [10]:
textos_pdf = {}

for archivo in descargados_nuevos:
    ruta = os.path.join(CARPETA_LOCAL, archivo)
    try:
        with pdfplumber.open(ruta) as pdf:
            paginas = [pagina.extract_text() or '' for pagina in pdf.pages]
            textos_pdf[archivo] = "\n".join(paginas)
        print(textos_pdf[archivo][:2000])  # Muestra los primeros 1000 caracteres
        print("="*80)
    except Exception as e:
        print(f"❌ Error al leer {archivo}: {e}")

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


RUT EMISOR: 215438030010
e-Ticket
SERIE NÚMERO PAGO MONEDA
A 12074 Crédito USD
RECEPTOR: 801012538
UYTECH SRL
UYTECH Soluciones IT
GLT CASA DE BOLSA S.A.
Maldonado 2002, Montevideo
Avda. Aviadores del Chaco No. 2050, World Trade Center Torre 4 Piso
24107513
ASUNCIÓN (PARAGUAY), PARAGUAY
FECHA 10/04/2025 VENCIMIENTO 10/05/2025 COTIZACIÓN 43.363
CÓDIGO DESCRIPCIÓN UN CANTIDAD UNITARIO DESCUENTO MONTO *
WIN_S25_CAL Windows Server 2025 RDS CAL Commercial Perpetual 10,00 211,14 2.111,40 1
TOTALES
Código de Seguridad ggIEAA Monto no Gravado 2.111,40
I.V.A. al día Monto IVA en Suspenso 0,00
CAE Nº 90232234960 Monto Neto IVA Tasa Mínima 0,00
Serie A Monto Neto IVA Tasa Básica 0,00
Rango 12001 al 13000 Importe IVA Tasa Mínima % 0,00 0,00
Fecha emisor 01/11/17 Importe IVA Tasa Básica % 0,00 0,00
Puede verificar comprobante en: Monto Total 2.111,40
z.uy/cfes Redondeo / No Facturable 0,00
Total a Pagar U$S 2.111,40
Hoja 1/1 Fecha de vencimiento de CAE 03/12/2025
ADENDA
Notas:
* 1.No gravado 2.Tasa

In [11]:
textos_pdf[archivo][:1200]

'RUT EMISOR: 215438030010\ne-Ticket\nSERIE NÚMERO PAGO MONEDA\nA 12074 Crédito USD\nRECEPTOR: 801012538\nUYTECH SRL\nUYTECH Soluciones IT\nGLT CASA DE BOLSA S.A.\nMaldonado 2002, Montevideo\nAvda. Aviadores del Chaco No. 2050, World Trade Center Torre 4 Piso\n24107513\nASUNCIÓN (PARAGUAY), PARAGUAY\nFECHA 10/04/2025 VENCIMIENTO 10/05/2025 COTIZACIÓN 43.363\nCÓDIGO DESCRIPCIÓN UN CANTIDAD UNITARIO DESCUENTO MONTO *\nWIN_S25_CAL Windows Server 2025 RDS CAL Commercial Perpetual 10,00 211,14 2.111,40 1\nTOTALES\nCódigo de Seguridad ggIEAA Monto no Gravado 2.111,40\nI.V.A. al día Monto IVA en Suspenso 0,00\nCAE Nº 90232234960 Monto Neto IVA Tasa Mínima 0,00\nSerie A Monto Neto IVA Tasa Básica 0,00\nRango 12001 al 13000 Importe IVA Tasa Mínima % 0,00 0,00\nFecha emisor 01/11/17 Importe IVA Tasa Básica % 0,00 0,00\nPuede verificar comprobante en: Monto Total 2.111,40\nz.uy/cfes Redondeo / No Facturable 0,00\nTotal a Pagar U$S 2.111,40\nHoja 1/1 Fecha de vencimiento de CAE 03/12/2025\nADENDA\n

## 🤖 Paso 4: Envío del texto a ChatGPT para análisis estructurado

- conexión con GPT-4 Turbo usando openai
- envío del texto del PDF
- recepción de una respuesta estructurada en formato JSON.

In [15]:
# Asignar la clave API desde la variable de entorno
api_key = os.getenv("OPENAI_API_KEY")

# Validar que la API Key exista
if not api_key:
    print("❌ Error: No se encontró la API Key de OpenAI. Revisa tu archivo .env.")
    sys.exit(1)

# Configurar OpenAI con la clave encontrada
openai.api_key = api_key

print("✅ API Key de OpenAI cargada correctamente.")

# Prompt de ejemplo y estructura deseada
system_prompt = "Sos un contador experto. Analiza la factura adjunta y extrae: fecha, proveedor, RUC o CUIT, monto total, tipo de gasto o ingreso, porcentaje de impuestos si existe."

assistant_prompt = (
    "Tu siguiente output devuelve solamente el formato JSON. Sin texto adicional\n"
    "Te muestro un ejemplo de tu output:\n"
    "[\n"
    "  {\n"
    "    \"fecha\": \"{{fecha_factura}}\",\n"
    "    \"proveedor\": \"{{nombre_proveedor}}\",\n"
    "    \"ruc_o_cuit\": \"{{ruc_o_cuit_proveedor}}\",\n"
    "    \"monto_total\": \"{{monto_total_factura}}\",\n"
    "    \"tipo\": \"{{tipo_gasto_o_ingreso}}\",\n"
    "    \"porcentaje_impuestos\": \"{{porcentaje_impuestos}}\"\n"
    "  }\n"
    "]"
)

user_prompt = f"Adjunto factura. Extrae y organiza la información contable.\n\n{texto}"

# Función que envía el texto del PDF a ChatGPT
def analizar_con_chatgpt(texto):
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "assistant", "content": assistant_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.2
    )
    
    # Obtener datos
    content = response.choices[0].message.content
    prompt_tokens = response.usage.prompt_tokens
    completion_tokens = response.usage.completion_tokens
    total_tokens = response.usage.total_tokens

    return content, prompt_tokens, completion_tokens, total_tokens

# Enviar el texto extraído del PDF de muestra
respuesta_chatgpt, prompt_tokens, completion_tokens, total_tokens = analizar_con_chatgpt(textos_pdf[archivo][:1200])

# Calcular el costo (para GPT-4o: $0.0005 input / $0.0015 output por cada 1000 tokens)
input_cost = (prompt_tokens / 1000000) * 0.60
output_cost = (completion_tokens / 1000000) * 2.40
total_cost = input_cost + output_cost

# Mostrar resultados
print("\n📦 Respuesta estructurada de ChatGPT:\n")
print(respuesta_chatgpt)

print("\n📊 Consumo de tokens:")
print(f"Tokens de entrada (prompt): {prompt_tokens}")
print(f"Tokens de salida (respuesta): {completion_tokens}")
print(f"Tokens totales: {total_tokens}")

print("\n💵 Costo estimado:")
print(f"Input cost: ${input_cost:.6f}")
print(f"Output cost: ${output_cost:.6f}")
print(f"Total cost: ${total_cost:.6f}")

✅ API Key de OpenAI cargada correctamente.

📦 Respuesta estructurada de ChatGPT:

[
  {
    "fecha": "10/04/2025",
    "proveedor": "UYTECH SRL",
    "ruc_o_cuit": "801012538",
    "monto_total": "2.111,40",
    "tipo": "gasto",
    "porcentaje_impuestos": "0"
  }
]

📊 Consumo de tokens:
Tokens de entrada (prompt): 629
Tokens de salida (respuesta): 75
Tokens totales: 704

💵 Costo estimado:
Input cost: $0.000377
Output cost: $0.000180
Total cost: $0.000557


In [55]:
def consultar_factura_chat(respuesta_chatgpt):
    while True:
        print("\n📝 Ingresá tu pregunta sobre la factura (o escribí 'salir' para terminar):")
        pregunta_usuario = input("> ")

        if pregunta_usuario.lower() == "salir":
            print("👋 Cerrando la consulta. ¡Hasta luego!")
            break

        # Hacer consulta a OpenAI
        response = openai.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "Sos un asistente contable. Respondé en forma breve, precisa y usando solamente los datos proporcionados."},
                {"role": "user", "content": f"Datos de la factura:\n{respuesta_chatgpt}\n\nPregunta: {pregunta_usuario}"}
            ],
            temperature=0
        )

        # Obtener datos de respuesta
        content = response.choices[0].message.content
        prompt_tokens = response.usage.prompt_tokens
        completion_tokens = response.usage.completion_tokens
        total_tokens = response.usage.total_tokens

        # Calcular el costo (usando tarifas de GPT-4o Mini)
        input_cost = (prompt_tokens / 1_000_000) * 0.60   # 0.60 USD por millón input
        output_cost = (completion_tokens / 1_000_000) * 2.40  # 2.40 USD por millón output
        total_cost = input_cost + output_cost

        # Mostrar resultados
        print("\n📋 Respuesta de ChatGPT:")
        print(content)

        print("\n📊 Consumo de tokens:")
        print(f"Tokens de entrada (prompt): {prompt_tokens}")
        print(f"Tokens de salida (respuesta): {completion_tokens}")
        print(f"Tokens totales: {total_tokens}")

        print("\n💵 Costo estimado:")
        print(f"Input cost: ${input_cost:.6f}")
        print(f"Output cost: ${output_cost:.6f}")
        print(f"Total cost: ${total_cost:.6f}")

# Llamada principal
consultar_factura_chat(respuesta_chatgpt)



📝 Ingresá tu pregunta sobre la factura (o escribí 'salir' para terminar):


>  cual fue el gasto de la ultima factura?



📋 Respuesta de ChatGPT:
El gasto de la última factura fue de 2.111,40.

📊 Consumo de tokens:
Tokens de entrada (prompt): 124
Tokens de salida (respuesta): 16
Tokens totales: 140

💵 Costo estimado:
Input cost: $0.000074
Output cost: $0.000038
Total cost: $0.000113

📝 Ingresá tu pregunta sobre la factura (o escribí 'salir' para terminar):


>  salir


👋 Cerrando la consulta. ¡Hasta luego!


## 📊 Paso 5: Exportación a Google Sheets

In [16]:
# Cambiar el alcance para incluir acceso a Sheets
SCOPES_SHEETS = ['https://www.googleapis.com/auth/spreadsheets']

# Reutilizar la credencial de cuenta de servicio
sheets_credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES_SHEETS
)
sheets_service = build('sheets', 'v4', credentials=sheets_credentials)

# ID de tu hoja de cálculo (copiado desde la URL de Google Sheets)
SPREADSHEET_ID = os.getenv('SPREADSHEET_ID')
SHEET_NAME = os.getenv('SHEET_NAME')

# Convertir respuesta JSON de ChatGPT a diccionario
datos_json = json.loads(respuesta_chatgpt)

# Verificar si datos_json es lista de diccionarios o solo un diccionario
if isinstance(datos_json, dict):
    datos_json = [datos_json]  # Lo convertimos en una lista de un elemento si era dict suelto

# Crear dataframe
df = pd.DataFrame(datos_json)

# Convertir a lista de listas para la API de Sheets
datos_a_escribir = [df.columns.tolist()] + df.values.tolist()

# Escribir en la hoja
sheets_service.spreadsheets().values().update(
    spreadsheetId=SPREADSHEET_ID,
    range=f"{SHEET_NAME}!A1",
    valueInputOption="RAW",
    body={"values": datos_a_escribir}
).execute()

print("✅ Datos exportados correctamente a Google Sheets")


✅ Datos exportados correctamente a Google Sheets


## 💬 Paso 6: Integración con WhatsApp + ChatGPT para consultas sobre Google Sheets
### _pending_

In [ ]:
# Este paso se implementa fuera del notebook como un backend con Flask/FastAPI expuesto vía HTTPS.
# Es necesario tener configurada la WhatsApp Cloud API desde Meta Developers.

# Flujo básico:
# 1. Usuario envía un mensaje por WhatsApp (ej: "¿Cuánto gasté en abril?")
# 2. El webhook lo recibe y llama a ChatGPT con un prompt armado dinámicamente en base a los datos del Google Sheet.
# 3. La respuesta es enviada de nuevo por WhatsApp al usuario.

# Requiere:
# - Cuenta en Meta Developers y configuración de WhatsApp Cloud API
# - Número de prueba o habilitado para producción
# - Token de acceso + configuración de Webhook (puede usarse ngrok para pruebas locales)
# - Flask + google-api-python-client + openai

# Código de ejemplo básico (Flask):
from flask import Flask, request
import openai
import json

app = Flask(__name__)
openai.api_key = 'TU_API_KEY_OPENAI'

@app.route('/webhook', methods=['POST'])
def webhook():
    mensaje = request.json['entry'][0]['changes'][0]['value']['messages'][0]['text']['body']
    # Aquí conectarías con Google Sheets y estructurarías el prompt
    prompt = f"Este es el contenido del mensaje recibido: '{mensaje}'. Buscá la respuesta en mis datos financieros."

    response = openai.ChatCompletion.create(
        model="gpt-4-turbo",
        messages=[
            {"role": "system", "content": "Sos un asistente que analiza información financiera en base a hojas de cálculo."},
            {"role": "user", "content": prompt}
        ]
    )
    respuesta = response["choices"][0]["message"]["content"]

    # Enviar la respuesta de nuevo a WhatsApp vía la API (omitir implementación específica aquí)
    print("Respuesta:", respuesta)
    return '', 200

# Este servidor debe estar desplegado en HTTPS para que WhatsApp lo acepte (ej: ngrok, VPS o cloud)
